In [42]:
import pandas as pd
from sklearn.cross_validation import KFold, cross_val_predict
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import scale
import time
import datetime
from tqdm import tqdm
import numpy as np

# Boosting

1 - Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [68]:
data = pd.read_csv('features.csv', index_col='match_id')
data.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16,2449,0,4,1974,3,63


In [3]:
full_x = data.ix[:,0:103]
full_y = data.ix[:,103:]

2 - Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [4]:
full_x.count()[full_x.count()<full_x.shape[0]]

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

3 - Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [5]:
x_0 = full_x.fillna(0)

In [68]:
#tmp = train_x[train_x['dire_flying_courier_time'].isnull()]

4 - Какой столбец содержит целевую переменную? Запишите его название.

In [6]:
target = full_y['radiant_win']

In [7]:
X = x_0.ix[:, 1:] #deleting 'start_time' feature

5 - Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [8]:
kf = KFold(target.size, n_folds=5, shuffle=True, random_state=241)


In [9]:
grid_estimators = [10,20,30]

In [27]:
for estimator in grid_estimators:
    
    start_time = datetime.datetime.now()
    
    clf = GradientBoostingClassifier(random_state=241, n_estimators=estimator)
    predict = cross_val_predict(clf, X, target, kf)
    score = roc_auc_score(target, predict)

    elapsed_time = datetime.datetime.now() - start_time

    print('n_estimators: {0}, time: {1}, ROC-AUC score: {2}'.format(estimator, elapsed_time, score))


n_estimators: 10, time: 0:00:47.718000, ROC-AUC score: 0.601568477717
n_estimators: 20, time: 0:01:44.625000, ROC-AUC score: 0.621090560945
n_estimators: 30, time: 0:03:13.501000, ROC-AUC score: 0.630319361785


### 1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?
1. Признаки события "первая кровь" (first blood). Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение: first_blood_time, first_blood_team, first_blood_player1, first_blood_player2 
2. Признаки приобретения предмета с префиксами rariant_ and dire_ могут иметь пропуски, если команда не получала указанные предметы в течение игры.

### 2. Как называется столбец, содержащий целевую переменную?
radiant_win

### 3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? Напомним, что в данном задании мы используем метрику качества AUC-ROC.

n_estimators: 20, time: 0:02:14.296000, ROC-AUC score: 0.621090560945

### 4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?

Смысл, безусловно, есть. При увеличении количества деревьев.
Что касается скорости, то стоит воспользоваться следующими советами из задания: 
Если все работает очень медлено:
- Используйте для обучения и кросс-валидации не всю выборку, а некоторое ее подмножество — например, половину объектов. Подмножество лучше всего брать случайным, а не формировать его из первых m объектов.
- Попробуйте упростить модель — например, уменьшить глубину деревьев в градиентом бустинге (max_depth).

# Logistic regression

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import  LogisticRegression

1 - Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [10]:
X_scaled = scale(X)

In [33]:
def get_best_score(X, y, kf):
    best_score = -1
    best_c = 0

    best_time = 0

    for c in tqdm(np.linspace(0.001, 1.5, num=8, endpoint=False)):
        print c
        start_time = datetime.datetime.now()

        clf_log = LogisticRegression(random_state=241, C=c, penalty='l2')
        predict_log = cross_val_predict(clf_log, X, y, kf)
        score_log = roc_auc_score(target, predict_log)

        elapsed_time = datetime.datetime.now() - start_time

        print('C: {0:}, time: {1} to fit, ROC-AUC score: {2}'.format(c, elapsed_time, score_log))

        if score_log > best_score:
            best_score = score_log
            best_c = c
            best_time = elapsed_time
    return best_score, best_c, best_time

In [14]:
best_score, best_c, best_time = get_best_score(X_scaled, target, kf)

  0%|                                                    | 0/8 [00:00<?, ?it/s]

0.001
C: 0.001, time: 0:00:09.078000 to fit, ROC-AUC score: 0.654145668482


 12%|█████▌                                      | 1/8 [00:09<01:03,  9.08s/it]

0.188375
C: 0.188375, time: 0:00:14.739000 to fit, ROC-AUC score: 0.654340232729


 25%|███████████                                 | 2/8 [00:23<01:04, 10.78s/it]

...

 75%|█████████████████████████████████           | 6/8 [01:30<00:30, 15.06s/it]

1.12525
C: 1.12525, time: 0:00:19.234000 to fit, ROC-AUC score: 0.65429979898


 88%|██████████████████████████████████████▌     | 7/8 [01:49<00:16, 16.31s/it]

1.312625
C: 1.312625, time: 0:00:16.230000 to fit, ROC-AUC score: 0.65429979898


100%|████████████████████████████████████████████| 8/8 [02:05<00:00, 16.29s/it]


In [16]:
print('Best C={0}; Time: {1}; Best ROC-AUC score: {2}'.format(best_c, best_time, best_score))

Best C=0.188375; Time: 0:00:14.739000; Best ROC-AUC score: 0.654340232729


2 - Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [35]:
cat_features1 = {
    'lobby_type'
}
cat_features2 = {
    'r1_hero',
    'r2_hero',
    'r3_hero',
    'r4_hero',
    'r5_hero',
    'd1_hero',
    'd2_hero',
    'd3_hero',
    'd4_hero',
    'd5_hero',
}

In [36]:
X_minus_cat1 = X.drop(cat_features1, axis=1)
X_minus_cat2 = X_minus_cat1.drop(cat_features2, axis=1)

In [23]:
X_scaled = scale(X_minus_cat2)

In [34]:
best_score, best_c, best_time = get_best_score(X_scaled, target, kf)

  0%|                                                    | 0/8 [00:00<?, ?it/s]

0.001
C: 0.001, time: 0:00:09.826000 to fit, ROC-AUC score: 0.654202117947


 12%|█████▌                                      | 1/8 [00:09<01:08,  9.83s/it]

0.188375
C: 0.188375, time: 0:00:14.810000 to fit, ROC-AUC score: 0.654523327945


 25%|███████████                                 | 2/8 [00:24<01:07, 11.32s/it]

...

 75%|█████████████████████████████████           | 6/8 [01:25<00:28, 14.12s/it]

1.12525
C: 1.12525, time: 0:00:14.585000 to fit, ROC-AUC score: 0.654512647836


 88%|██████████████████████████████████████▌     | 7/8 [01:39<00:14, 14.26s/it]

1.312625
C: 1.312625, time: 0:00:14.674000 to fit, ROC-AUC score: 0.654512647836


100%|████████████████████████████████████████████| 8/8 [01:54<00:00, 14.39s/it]


In [37]:
print('Best C={0}; Time: {1}; Best ROC-AUC score: {2}'.format(best_c, best_time, best_score))

Best C=0.188375; Time: 0:00:14.810000; Best ROC-AUC score: 0.654523327945


3 - На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [91]:
def get_unique_heroes(data):
    unique_heroes = set()

    for name in cat_features2:
        unique_heroes.update(data[name].value_counts().index.values)

    return list(unique_heroes)

unique_heroes = get_unique_heroes(data)
print "Unique heroes count: ", len(unique_heroes)

Unique heroes count:  108


4 - Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа

In [103]:
def get_bag_of_words(X, unique_heroes):
    
    X_pick = np.zeros((X.shape[0], len(unique_heroes)))

    for i, match_id in enumerate(X.index):
        for p in xrange(5):
            r = X.ix[match_id, 'r%d_hero' % (p+1)]
            X_pick[i, unique_heroes.index(r)] = 1

            d = X.ix[match_id, 'd%d_hero' % (p+1)]
            X_pick[i, unique_heroes.index(d)] = -1
    return X_pick

In [104]:
X_pick = get_bag_of_words(X, unique_heroes)

5 - Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [76]:
X_scaled = np.hstack((X_scaled, X_pick))

In [80]:
best_score, best_c, best_time = get_best_score(X_scaled, target, kf)

  0%|                                                    | 0/8 [00:00<?, ?it/s]

0.001
C: 0.001, time: 0:00:11.873000 to fit, ROC-AUC score: 0.679030990623


 12%|█████▌                                      | 1/8 [00:11<01:23, 11.87s/it]

0.188375
C: 0.188375, time: 0:00:30.229000 to fit, ROC-AUC score: 0.682437272257


 25%|███████████                                 | 2/8 [00:42<01:44, 17.39s/it]

...

 75%|█████████████████████████████████           | 6/8 [03:13<01:08, 34.06s/it]

1.12525
C: 1.12525, time: 0:00:39.133000 to fit, ROC-AUC score: 0.682409043075


 88%|██████████████████████████████████████▌     | 7/8 [03:52<00:35, 35.59s/it]

1.312625
C: 1.312625, time: 0:00:45.413000 to fit, ROC-AUC score: 0.682397600737


100%|████████████████████████████████████████████| 8/8 [04:37<00:00, 38.54s/it]


In [81]:
print('Best C: {0}; Time: {1}, Best ROC-AUC score: {2}'.format(best_c, best_time, best_score))

Best C: 0.188375; Time: 0:00:30.229000, Best ROC-AUC score: 0.682437272257


##### score: 0.682437272257 - приятный отрыв. возьмём последнюю модель

6 - Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [118]:
test_data = pd.read_csv('features_test.csv', index_col='match_id')
test_data.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12,247,-86,272,3,4,2,0,118
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29,168,-54,NaN,3,2,2,1,16
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22,46,-87,186,1,3,3,0,-34
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49,30,-89,210,3,4,2,1,-26
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36,180,-86,180,1,3,2,1,-33


In [119]:
full_x_test = test_data.ix[:,0:103]
full_y_test = test_data.ix[:,103:]
full_x_test.count()[full_x_test.count()<full_x_test.shape[0]]

first_blood_time               13625
first_blood_team               13625
first_blood_player1            13625
first_blood_player2             9411
radiant_bottle_time            14282
radiant_courier_time           17050
radiant_flying_courier_time    12292
radiant_first_ward_time        16847
dire_bottle_time               14335
dire_courier_time              17047
dire_flying_courier_time       12653
dire_first_ward_time           16914
dtype: int64

In [120]:
x_test_0 = full_x_test.fillna(0)
unique_heroes_test = get_unique_heroes(test_data)
X_pick_test = get_bag_of_words(x_test_0, unique_heroes_test) 

In [121]:
x_test = x_test_0.drop(cat_features1, axis=1)
x_test = x_test.drop(cat_features2, axis=1)

In [123]:
X = x_test.ix[:, 1:] #deleting 'start_time' feature

In [125]:
x_test_scaled = scale(x_test)
x_test_scaled = np.hstack((x_test_scaled, X_pick_test))

In [126]:
best_clf = LogisticRegression(penalty='l2', C=best_c)
best_clf.fit(X_scaled, target)

LogisticRegression(C=0.18837500000000001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [128]:
Y_predict = best_clf.predict(x_test_scaled)

In [41]:
df = pd.DataFrame(index=x_test.index, columns=['radiant_win'])
df['radiant_win'] = best_clf.predict_proba(x_test_scaled)
df.to_csv('predictions.csv')

In [44]:
print min(df['radiant_win'])
print max(df['radiant_win'])

0.0185172568072
0.983184479968


### 1 - Какое качество получилось у логистической регрессии над всеми исходными признаками? 
0.6543

### Как оно соотносится с качеством градиентного бустинга? 
больше (у бустинга 0.621090560945)

### Чем вы можете объяснить эту разницу? 
тем, что было построено недостоточно деревьев. Было бы больше деревьев -- качество у бцстинга было бы выше. При этом алгоритм обучался бы дольше. Так что логистическая регрессия тут хороша.

### Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

Да, быстрее. См. выше.

### 2 - Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?

Качество вырастает (score: 0.6545), поскольку категориальные признаки используются не в лоб, а только после их правильного преобразования


### 3 - Сколько различных идентификаторов героев существует в данной игре?

108

### 4 - Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?

Улучшилось (score: 0.682437272257). В любом случае, категориальные признаки имеют вес и их использование только помогает делать прогноз. Мешок слов -- отличный способ закодировать данный набор категориальных признаков.

### 5 - Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

0.0185172568072
0.983184479968

In [83]:
#мур